#1. Data

In [ ]:
import tensorflow as tf
import tf_keras as keras

import numpy as np

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print(tf.__version__)
print(hub.__version__)
print(keras.__version__)
print('GPU', 'On' if tf.config.experimental.list_physical_devices("GPU") else 'Off')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 68.7 MB/s eta 0:00:00
2.18.0
0.16.1
2.18.0
GPU On


In [ ]:
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.2SET4C_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.2SET4C_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.2SET4C_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
len(train_data), len(validation_data), len(test_data)

(15000, 10000, 25000)

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

#2. Model

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:5])

<tf.Tensor: shape=(5, 20), dtype=float32, numpy=
array([[ 1.7657859 , -3.882232  ,  3.913424  , -1.5557289 , -3.3362343 ,
        -1.7357956 , -1.9954445 ,  1.298955  ,  5.081597  , -1.1041285 ,
        -2.0503852 , -0.7267516 , -0.6567596 ,  0.24436145, -3.7208388 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489715 , -1.1315986 ],
       [ 1.8804485 , -2.5852385 ,  3.4066994 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.7855542 ,  1.3874227 ,  3.8476458 , -0.9256539 ,
        -1.896706  ,  1.2113281 ,  0.11474716,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015903 , -1.6390051 ],
       [ 0.71152216, -0.63532174,  1.7385626 , -1.1168287 , -0.54515934,
        -1.1808155 ,  0.09504453,  1.4653089 ,  0.66059506,  0.79308075,
        -2.2268343 ,  0.07446616, -1.4075902 , -0.706454  , -1.907037  ,
         1.4419788 ,  1.9551864 , -0.42660046, -2.8022065 ,  0.43727067],
       [ 1.5164999 , -0.7103405 ,  1.8556769 , -1.2033532 , -1.3105817 ,

In [ ]:
model = keras.Sequential()
#hub_layer = tf.keras.layers.Lambda(hub_layer)
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/tf_keras/src/backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


30/30 [==============================] - 10s 185ms/step - loss: 0.8491 - accuracy: 0.5061 - val_loss: 0.6889 - val_accuracy: 0.5860
Epoch 2/20
30/30 [==============================] - 6s 189ms/step - loss: 0.6448 - accuracy: 0.6321 - val_loss: 0.6155 - val_accuracy: 0.6628
Epoch 3/20
30/30 [==============================] - 5s 161ms/step - loss: 0.5898 - accuracy: 0.6955 - val_loss: 0.5773 - val_accuracy: 0.7090
Epoch 4/20
30/30 [==============================] - 6s 183ms/step - loss: 0.5500 - accuracy: 0.7353 - val_loss: 0.5413 - val_accuracy: 0.7431
Epoch 5/20
30/30 [==============================] - 4s 144ms/step - loss: 0.5115 - accuracy: 0.7705 - val_loss: 0.5075 - val_accuracy: 0.7696
Epoch 6/20
30/30 [==============================] - 8s 268ms/step - loss: 0.4727 - accuracy: 0.8013 - val_loss: 0.4740 - val_accuracy: 0.7925
Epoch 7/20
30/30 [==============================] - 4s 134ms/step - loss: 0.4346 - accuracy: 0.8266 - val_loss: 0.4427 - val_accuracy: 0.8159
Epoch 8/20
30/30

In [ ]:
results = model.evaluate(test_data.batch(512))

for metrics, value in zip(model.metrics_names, results):
    print("Metrics: %s Value: %.3f" % (metrics, value))

49/49 [==============================] - 2s 47ms/step - loss: 0.3062 - accuracy: 0.8703
Metrics: loss Value: 0.306
Metrics: accuracy Value: 0.870
